# Application Architecture Evolution

<img src="img/Snip20191118_40.png" width=80%/>

- mainframe (1960s ~ 1970s)
- personal computer era (1980s)
- web era (1990s onwards)

# SQL Injection

- sql injection is a code injection technique, used to attack data-driven applications, in which nefarious SQL statements are inserted into an entry field for execution (e.g., to dump the db contents to the attacker)


- **first-order SQL Injection**: caused by the application trusting data obtained directly from the user
    - e.g., `SELECT * FROM instructor WHERE name = "' + name '"`
    - users enters `x' or 'y' = 'y`
    - the query actually executed is equivalent to `SELECT * FROM instructor`

- **second-order SQL injection**: queries may be vulnerable to these even if the variable data comes from an existing database table
    - `name = result of "SELECT name FROM instructor ORDER BY salary LIMIT 1"`
    - `SELECT dept_name FROM instructor WHERE name = "' + name '"`
    - harder to carry out successfully because it requires a deeper understanding of data flow within the application



# SQL Injection Prevention

- in application code, **never concatenate strings to form an SQL query**
    - always use a **prepared statement** to incorporate user input into a query


```java
Connection conn = DriverManager.getConnection(url, user, pwd)
PreparedStatement pStmt = conn.prepareStatement("SELECT * FROM instructor WHERE ID = ?")
pStmt.setInt(1, dataFromWebForm);
ResultSet rs = pStmt.executeQuery();
```

- prepared statements ensure that user input is treated as data and not executable SQL code

- escaping characters only take you so far
    - it is unclear what characters are unsafe
    - DB/API can deal with different encodings

- in SQL, stored procedure can integrate inputs safely into a query without to do string concatenation

# Authentication

- the process of determining whether someone or something is, in fact, who or what it is declared to be
- can be performed by the application or by the DBMS
    - common to use both
    - the DB admin configures one DB user for each web app user the **create user** and **grant** SQL commands (DCL)
    - the application keeps track of human users by storing logins and passwords in a table
    - the application authenticates the human against the application-maintained user table; the DB authenticates the web application against its own user table

# Password Authentication

- never store plaintext passwords
- Store a **hash** of the password: `INSERT INTO Users VALUES (login, SHA2(password, 512))`
    - to check password, compare the hash of user-provided password against the hash stored in the application-maintained password table
- Combine the password with a random **salt** before hashing `INSERT INTO Users VALUES (login, salt, SHA2(CONCAT(salt, password), 512))`

## how it works
- hashing alone prevents an attacker from seeing the plaintext password if they hack the DBMS or the host
- salting prevents an attacker from cracking a weak password quickly using a pre-computed "rainbow table"
- since the salts are stored in plaintext, an attacker can still crack a weak password using brute force

## tips
- always compute the hash at the server, never at the client
- use a good cryptographic hash function such as **SHA-2** or a key derivation function such as **bcrypt** or **PBKDF2**
- use a large random salt (e.g., same size as the hash)
- use a different salt for each user, and re-generate the salt each time the password is changed


# Access Control

- selective restriction of **access** to a place or resource
- **role-based access control** in web applications
    - define a role for each user
    - for each component of the web application, decide which users should have access
    - each time a component is accessed check that the user's role allows it
    - do not assume that a user should have access to a component just because they know the URL
    - managed entirely by the application
    